In [102]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.metrics import accuracy_score

In [116]:
data = {
    'Text': [
        "Компьютер не включается, горит красный индикатор на материнской плате.",
        "Компьютер перегревается и автоматически выключается через 10 минут работы.",
        "Монитор не показывает изображение, но компьютер включается.",
        "Компьютер работает медленно и долго загружается.",
        "Жесткий диск издает странные звуки.",
        "Экран черный, но кулеры работают.",
        "При загрузке система зависает на логотипе.",
        "Не работает клавиатура и мышь.",
        "Нет звука на компьютере.",
        "Видеокарта перегревается и вызывает артефакты."
    ],
    'Label1': ['Питание', 'Охлаждение', 'Монитор', 'Производительность', 
               'Жесткий диск', 'Монитор', 'Процессор', 'Периферия', 
               'Аудио', 'Видеокарта'],
    'Label2': ['Материнская плата', 'Процессор', 'Видеокарта', 
               'Операционная система', 'Жесткий диск', 'Монитор', 
               'Материнская плата', 'Периферия', 'Звуковая карта', 
               'Видеокарта']
}

In [117]:
df = pd.DataFrame(data)
texts = df['Text']
labels = pd.get_dummies(df[['Label1', 'Label2']])

In [127]:
import re

STOP_WORDS = {"не", "после", "но", "на", "в", "и", "горит", "нескольких", "минут"}

def tokenize(text, stopwords):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords]
    return tokens

def build_vocabulary(texts):
    vocab = set()
    for text in texts:
        tokens = tokenize(text, STOP_WORDS)
        vocab.update(tokens)
    return list(vocab)

def text_to_vector(text, vocabulary):
    tokens = tokenize(text, STOP_WORDS)
    vector = [1 if word in tokens else 0 for word in vocabulary]
    return vector

vocabulary = build_vocabulary(texts)

vectors = [text_to_vector(text, vocabulary) for text in texts]

X = [text_to_vector(text, vocabulary) for text in texts]
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(labels.values, dtype=torch.long)

In [128]:
class ImprovedNN(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_dim):
        super(ImprovedNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p=0.3)
        self.fc3 = nn.Linear(hidden_dim2, hidden_dim3)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_dim3, output_dim)

    def forward(self, x):
        x = self.dropout1(self.relu1(self.fc1(x)))
        x = self.dropout2(self.relu2(self.fc2(x)))
        x = self.relu3(self.fc3(x))
        x = self.fc4(x)
        return x

In [129]:
def train_model(model, optimizer, criterion, X_train, y_train, epochs=10):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
    return model

In [130]:
def evaluate_model(model, X_val, y_val):
    model.eval()
    with torch.no_grad():
        outputs = model(X_val)
        predicted = torch.argmax(outputs, dim=1)
        accuracy = accuracy_score(y_val, predicted)
    return accuracy

In [144]:
param_grid = {
    'lr': [0.001, 0.01, 0.1],
    'hidden_dim1': [128, 256],
    'hidden_dim2': [64, 128],
    'hidden_dim3': [32, 64],
    'epochs': [100, 200]
}

In [145]:
input_dim = X.shape[1]
output_dim = y.shape[1]

In [146]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y.argmax(axis=1), test_size=0.3, random_state=42
)

In [147]:
best_params = None
best_accuracy = 0

for lr in param_grid['lr']:
    for h1 in param_grid['hidden_dim1']:
        for h2 in param_grid['hidden_dim2']:
            for h3 in param_grid['hidden_dim3']:
                for epochs in param_grid['epochs']:
                    model = ImprovedNN(input_dim, h1, h2, h3, output_dim)
                    optimizer = optim.Adam(model.parameters(), lr=lr)
                    criterion = nn.CrossEntropyLoss()
                    
                    model = train_model(model, optimizer, criterion, X_train, y_train, epochs=epochs)
                    
                    accuracy = evaluate_model(model, X_val, y_val)
                    print(f"Текущие параметры: lr={lr}, h1={h1}, h2={h2}, h3={h3}, epochs={epochs}, acc={accuracy:.4f}")
                    
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_params = {'lr': lr, 'hidden_dim1': h1, 'hidden_dim2': h2, 'hidden_dim3': h3, 'epochs': epochs}
                        

print(f"\nЛучшие параметры: {best_params} с точностью {best_accuracy:.4f}")

Текущие параметры: lr=0.001, h1=128, h2=64, h3=32, epochs=100, acc=0.0000
Текущие параметры: lr=0.001, h1=128, h2=64, h3=32, epochs=200, acc=0.0000
Текущие параметры: lr=0.001, h1=128, h2=64, h3=64, epochs=100, acc=0.0000
Текущие параметры: lr=0.001, h1=128, h2=64, h3=64, epochs=200, acc=0.0000
Текущие параметры: lr=0.001, h1=128, h2=128, h3=32, epochs=100, acc=0.0000
Текущие параметры: lr=0.001, h1=128, h2=128, h3=32, epochs=200, acc=0.0000
Текущие параметры: lr=0.001, h1=128, h2=128, h3=64, epochs=100, acc=0.0000
Текущие параметры: lr=0.001, h1=128, h2=128, h3=64, epochs=200, acc=0.0000
Текущие параметры: lr=0.001, h1=256, h2=64, h3=32, epochs=100, acc=0.0000
Текущие параметры: lr=0.001, h1=256, h2=64, h3=32, epochs=200, acc=0.0000
Текущие параметры: lr=0.001, h1=256, h2=64, h3=64, epochs=100, acc=0.0000
Текущие параметры: lr=0.001, h1=256, h2=64, h3=64, epochs=200, acc=0.0000
Текущие параметры: lr=0.001, h1=256, h2=128, h3=32, epochs=100, acc=0.0000
Текущие параметры: lr=0.001, h1=2

In [148]:
best_model = ImprovedNN(
    input_dim=input_dim, 
    hidden_dim1=best_params['hidden_dim1'], 
    hidden_dim2=best_params['hidden_dim2'], 
    hidden_dim3=best_params['hidden_dim3'], 
    output_dim=output_dim
)

optimizer = optim.Adam(best_model.parameters(), lr=best_params['lr'])
criterion = nn.CrossEntropyLoss()

train_model(best_model, optimizer, criterion, X_train, y_train, epochs=best_params['epochs'])

ImprovedNN(
  (fc1): Linear(in_features=41, out_features=256, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (relu2): ReLU()
  (dropout2): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=64, out_features=17, bias=True)
)

In [149]:
new_texts = [
    "Компьютер выключается через несколько минут работы.",
    "Жесткий диск странно щелкает.",
    "Монитор перестал показывать изображение."
]

new_text_vectors = [text_to_vector(text, vocabulary) for text in new_texts]
new_text_tensor = torch.tensor(new_text_vectors, dtype=torch.float32)

best_model.eval()
with torch.no_grad():
    predictions = best_model(new_text_tensor)
    predicted_classes = torch.argmax(predictions, dim=1)

for text, predicted_class in zip(new_texts, predicted_classes):
    predicted_label = labels.columns[predicted_class.item()]
    print(f"Текст: \"{text}\" => Предсказанная категория: {predicted_label}")

Текст: "Компьютер выключается через несколько минут работы." => Предсказанная категория: Label1_Производительность
Текст: "Жесткий диск странно щелкает." => Предсказанная категория: Label1_Жесткий диск
Текст: "Монитор перестал показывать изображение." => Предсказанная категория: Label1_Монитор
